<a href="https://colab.research.google.com/github/sinamahdavi/aml-2025-mistake-detection/blob/step2/notebooks/colab_compare_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Colab Quickstart

In [8]:
%cd /content
!rm -rf code

!git clone --recursive -b step2 https://github.com/sinamahdavi/aml-2025-mistake-detection.git code
%cd code

/content
Cloning into 'code'...
remote: Enumerating objects: 547, done.
remote: Counting objects: 100% (547/547), done.
remote: Compressing objects: 100% (222/222), done.
remote: Total 547 (delta 359), reused 498 (delta 318), pack-reused 0 (from 0)
Receiving objects: 100% (547/547), 255.62 KiB | 2.01 MiB/s, done.
Resolving deltas: 100% (359/359), done.
Submodule 'annotations' (https://github.com/CaptainCook4D/annotations) registered for path 'annotations'
Cloning into '/content/code/annotations'...
remote: Enumerating objects: 152, done.        
remote: Counting objects: 100% (152/152), done.        
remote: Compressing objects: 100% (98/98), done.        
remote: Total 152 (delta 75), reused 108 (delta 46), pack-reused 0 (from 0)        
Receiving objects: 100% (152/152), 793.14 KiB | 3.22 MiB/s, done.
Resolving deltas: 100% (75/75), done.
Submodule path 'annotations': checked out '0e9a108be2cbcbcbd592e7418c0ab9c16232d27a'
/content/code


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
# Setup data and checkpoints
# TODO: Update paths below with your own Google Drive paths

import os
os.chdir('/content/code')

# Create directories
!mkdir -p data/video/omnivore
!mkdir -p checkpoints

# Extract features (update path to your omnivore features zip)
print("Extracting features...")
if os.path.exists("/content/drive/MyDrive/AML/data/features/omnivore.zip"):
    # Extract to data/video/omnivore
    !unzip -q "/content/drive/MyDrive/AML/data/features/omnivore.zip" -d data/video/ 2>&1

    # If files were extracted to data/video/omnivore/omnivore/, move them up
    if os.path.exists("data/video/omnivore/omnivore"):
        !mv data/video/omnivore/omnivore/* data/video/omnivore/ 2>/dev/null || true
        !rmdir data/video/omnivore/omnivore 2>/dev/null || true

    # Verify extraction
    if os.path.exists("data/video/omnivore"):
        num_files = len([f for f in os.listdir("data/video/omnivore") if f.endswith('.npz')])
        print(f"✅ Extracted {num_files} feature files to data/video/omnivore/")
    else:
        print("⚠️  Extraction may have failed. Check the zip structure.")
else:
    print("⚠️  Features zip not found. Please update the path.")
    print("   Expected: /content/drive/MyDrive/AML/data/features/omnivore.zip")

# Extract checkpoints if needed (optional)
# if os.path.exists("/content/drive/MyDrive/AML/code/error_recognition_best.zip"):
#     !unzip -q "/content/drive/MyDrive/AML/code/error_recognition_best.zip" -d checkpoints/ 2>&1
#     print("✅ Checkpoints extracted")

Extracting features...
✅ Extracted 384 feature files to data/video/omnivore/


In [4]:
!pip install torcheval tabulate
!pip install loguru

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.2/179.2 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 3.5 MB/s eta 0:00:00


In [5]:
# Verify annotations submodule and check CUDA availability
import os
import torch
os.chdir('/content/code')

# Check CUDA availability
print("=" * 60)
print("DEVICE CHECK")
print("=" * 60)
if torch.cuda.is_available():
    print(f"✅ CUDA available: {torch.cuda.get_device_name(0)}")
    print(f"   CUDA Version: {torch.version.cuda}")
else:
    print("⚠️  CUDA not available - will use CPU")
    print("   Note: Enable GPU in Colab: Runtime → Change runtime type → GPU")
print("=" * 60)

# Check if annotations directory is properly initialized
if not os.path.exists('annotations/annotation_json/error_annotations.json'):
    print("\n📦 Initializing annotations submodule...")
    !git submodule update --init --recursive
    print("✅ Annotations submodule initialized")
else:
    print("\n✅ Annotations already initialized")

DEVICE CHECK
✅ CUDA available: Tesla T4
   CUDA Version: 12.6

✅ Annotations already initialized


In [15]:
# Train MLP model for error recognition
import os
os.chdir('/content/code')

!python train_er.py \
    --variant MLP \
    --backbone omnivore \
    --split recordings \
    --batch_size 8 \
    --num_epochs 10 \
    --lr 1e-3 \
    --weight_decay 1e-3

-------------------------------------------------------------
Training step model and testing on step level
Train args: {'num_workers': 8, 'pin_memory': False, 'shuffle': True, 'batch_size': 8}
Test args: {'num_workers': 8, 'pin_memory': False, 'shuffle': False, 'batch_size': 1}
{'batch_size': 8, 'test_batch_size': 1, 'num_epochs': 10, 'lr': 0.001, 'weight_decay': 0.001, 'ckpt': None, 'seed': 42, 'backbone': 'omnivore', 'ckpt_directory': './checkpoints', 'split': 'recordings', 'variant': 'MLP', 'model_name': None, 'task_name': 'error_recognition', 'error_category': None, 'modality': ['video'], 'device': None}
-------------------------------------------------------------
Loaded annotations...... 
Loading recording ids from recordings_combined_splits.json
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this

In [16]:
# Train Transformer model for error recognition
import os
os.chdir('/content/code')

!python train_er.py \
    --variant Transformer \
    --backbone omnivore \
    --split recordings \
    --batch_size 8 \
    --num_epochs 10 \
    --lr 1e-3 \
    --weight_decay 1e-3

-------------------------------------------------------------
Training step model and testing on step level
Train args: {'num_workers': 8, 'pin_memory': False, 'shuffle': True, 'batch_size': 8}
Test args: {'num_workers': 8, 'pin_memory': False, 'shuffle': False, 'batch_size': 1}
{'batch_size': 8, 'test_batch_size': 1, 'num_epochs': 10, 'lr': 0.001, 'weight_decay': 0.001, 'ckpt': None, 'seed': 42, 'backbone': 'omnivore', 'ckpt_directory': './checkpoints', 'split': 'recordings', 'variant': 'Transformer', 'model_name': None, 'task_name': 'error_recognition', 'error_category': None, 'modality': ['video'], 'device': None}
-------------------------------------------------------------
Loaded annotations...... 
Loading recording ids from recordings_combined_splits.json
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than w

In [11]:
# Train LSTM model for error recognition
import os
os.chdir('/content/code')

!python train_lstm.py \
    --variant LSTM \
    --backbone omnivore \
    --split recordings \
    --batch_size 8 \
    --num_epochs 10 \
    --lr 1e-3 \
    --weight_decay 1e-3

Training LSTM model for Error Recognition (Step 2b)
Backbone: omnivore
Split: recordings
Learning Rate: 0.001
Epochs: 10
Device: cuda
Loaded annotations...... 
Loading recording ids from recordings_combined_splits.json
Loaded annotations...... 
Loading recording ids from recordings_combined_splits.json
Loaded annotations...... 
Loading recording ids from recordings_combined_splits.json
Train Epoch: 1, Progress: 496/497, Loss: 0.717343: 100% 497/497 [00:34<00:00, 14.26it/s]
val Progress: 681/86: 100% 86/86 [00:04<00:00, 19.32it/s]
----------------------------------------------------------------
val Sub Step Level Metrics: {'precision': 0.3494809688581315, 'recall': 0.8595744680851064, 'f1': 0.4969249692496925, 'accuracy': 0.39941262848751835, 'auc': np.float64(0.5154422287949624), 'pr_auc': tensor(0.3489)}
val Step Level Metrics: {'precision': 0.07894736842105263, 'recall': 0.42857142857142855, 'f1': 0.13333333333333333, 'accuracy': 0.5465116279069767, 'auc': np.float64(0.39963833634719

In [12]:
# Evaluate LSTM model
import os
import glob
os.chdir('/content/code')

# Find the best checkpoint
lstm_ckpts = glob.glob('checkpoints/error_recognition/LSTM/omnivore/*_best.pt')
if not lstm_ckpts:
    # If no _best.pt, find latest checkpoint
    all_ckpts = sorted(glob.glob('checkpoints/error_recognition/LSTM/omnivore/*.pt'), key=os.path.getmtime)
    if all_ckpts:
        lstm_ckpt = all_ckpts[-1]  # Latest checkpoint
    else:
        raise FileNotFoundError("No LSTM checkpoints found!")
else:
    lstm_ckpt = lstm_ckpts[0]

print(f"Using checkpoint: {lstm_ckpt}")

# Evaluate on recordings split
!python -m core.evaluate_error_types \
    --variant LSTM \
    --backbone omnivore \
    --split recordings \
    --ckpt "{lstm_ckpt}" \
    --threshold 0.4 \
    --save_csv

Using checkpoint: checkpoints/error_recognition/LSTM/omnivore/error_recognition_recordings_omnivore_LSTM_video_best.pt
Using device: cuda
Loaded checkpoint: checkpoints/error_recognition/LSTM/omnivore/error_recognition_recordings_omnivore_LSTM_video_best.pt
Loaded annotations for error type analysis...
Evaluating: 100% 671/671 [00:04<00:00, 158.53it/s]

ERROR TYPE ANALYSIS - LSTM (omnivore) - recordings split

OVERALL METRICS:
----------------------------------------
  Accuracy:  45.16%
  Precision: 38.43%
  Recall:    87.55%
  F1 Score:  53.42%
  AUC:       57.48%

PER ERROR TYPE METRICS:
--------------------------------------------------------------------------------
Error Type                   Count      Acc     Prec   Recall       F1      AUC
--------------------------------------------------------------------------------
Technique Error                  0    0.00%    0.00%    0.00%    0.00%    0.00%
Preparation Error                0    0.00%    0.00%    0.00%    0.00%    0.00%
T

In [17]:
# Compare all three baselines
import os
import glob
os.chdir('/content/code')

# Find checkpoints for all models
def find_best_ckpt(pattern):
    best = glob.glob(pattern.replace('*.pt', '*_best.pt'))
    if best:
        return best[0]
    all_ckpts = sorted(glob.glob(pattern), key=os.path.getmtime)
    return all_ckpts[-1] if all_ckpts else None

mlp_ckpt = find_best_ckpt('checkpoints/error_recognition/MLP/omnivore/*.pt')
transformer_ckpt = find_best_ckpt('checkpoints/error_recognition/Transformer/omnivore/*.pt')
lstm_ckpt = find_best_ckpt('checkpoints/error_recognition/LSTM/omnivore/*.pt')

print("=" * 70)
print("CHECKPOINT SEARCH RESULTS")
print("=" * 70)
print(f"MLP: {mlp_ckpt if mlp_ckpt else '❌ Not found'}")
print(f"Transformer: {transformer_ckpt if transformer_ckpt else '❌ Not found'}")
print(f"LSTM: {lstm_ckpt if lstm_ckpt else '❌ Not found'}")
print("=" * 70)

# Build command with only available checkpoints
checkpoints = {}
if mlp_ckpt and os.path.exists(mlp_ckpt):
    checkpoints['MLP'] = mlp_ckpt
if transformer_ckpt and os.path.exists(transformer_ckpt):
    checkpoints['Transformer'] = transformer_ckpt
if lstm_ckpt and os.path.exists(lstm_ckpt):
    checkpoints['LSTM'] = lstm_ckpt

if len(checkpoints) == 0:
    print("\n❌ No checkpoints found! Please train models first.")
    print("   Available checkpoints will be compared automatically.")
elif len(checkpoints) == 1:
    print(f"\n⚠️  Only {list(checkpoints.keys())[0]} checkpoint found.")
    print("   Need at least 2 models to compare. Train other models first.")
else:
    print(f"\n✅ Found {len(checkpoints)} checkpoints. Comparing...")

    # Build command dynamically using subprocess
    import subprocess
    cmd = ["python", "compare_baselines.py", "--split", "recordings", "--backbone", "omnivore"]
    if 'MLP' in checkpoints:
        cmd.extend(["--mlp_ckpt", checkpoints["MLP"]])
    if 'Transformer' in checkpoints:
        cmd.extend(["--transformer_ckpt", checkpoints["Transformer"]])
    if 'LSTM' in checkpoints:
        cmd.extend(["--lstm_ckpt", checkpoints["LSTM"]])
    cmd.append("--save_csv")

    result = subprocess.run(cmd, capture_output=True, text=True)
    print(result.stdout)
    if result.stderr:
        print("Errors:", result.stderr)

CHECKPOINT SEARCH RESULTS
MLP: checkpoints/error_recognition/MLP/omnivore/error_recognition_recordings_omnivore_MLP_video_best.pt
Transformer: checkpoints/error_recognition/Transformer/omnivore/error_recognition_recordings_omnivore_Transformer_video_best.pt
LSTM: checkpoints/error_recognition/LSTM/omnivore/error_recognition_recordings_omnivore_LSTM_video_best.pt

✅ Found 3 checkpoints. Comparing...
Using device: cuda

📊 Evaluating MLP...
Loaded annotations...... 
Loading recording ids from recordings_combined_splits.json

📊 Evaluating Transformer...
Loaded annotations...... 
Loading recording ids from recordings_combined_splits.json

📊 Evaluating LSTM...
Loaded annotations...... 
Loading recording ids from recordings_combined_splits.json

BASELINE COMPARISON - RECORDINGS SPLIT
+-------------+------------+------------+-------------+----------+-------+-------+
| Model       | Backbone   |   Accuracy |   Precision |   Recall |    F1 |   AUC |
+=============+============+============+=====

In [21]:
# Display results
import pandas as pd
import os

# Show baseline comparison
comparison_path = 'results/baseline_comparison.csv'
if os.path.exists(comparison_path):
    df = pd.read_csv(comparison_path)
    print("=" * 70)
    print("BASELINE COMPARISON RESULTS")
    print("=" * 70)
    print(df.to_string(index=False))
else:
    print(f"Results file not found: {comparison_path}")

# Show error type analysis
error_type_path = 'results/error_type_analysis/LSTM_omnivore_recordings_error_type_analysis.csv'
if os.path.exists(error_type_path):
    print("\n" + "=" * 70)
    print("LSTM ERROR TYPE ANALYSIS")
    print("=" * 70)

    # The CSV has two sections, so read it manually
    try:
        with open(error_type_path, 'r') as f:
            lines = f.readlines()

        # Find where "Per Error Type Metrics" section starts
        per_error_start = None
        for i, line in enumerate(lines):
            if 'Per Error Type Metrics' in line:
                per_error_start = i + 1
                break

        if per_error_start:
            # Read from the per-error-type section (skip header line)
            df_errors = pd.read_csv(error_type_path, skiprows=per_error_start + 1)
            print(df_errors.to_string(index=False))
        else:
            # Fallback: display raw content
            print("".join(lines))
    except Exception as e:
        print(f"Error reading CSV: {e}")
        # Fallback: display raw content
        with open(error_type_path, 'r') as f:
            print(f.read())
else:
    print(f"\nError type analysis not found: {error_type_path}")


BASELINE COMPARISON RESULTS
      Model Backbone      Split  Accuracy  Precision  Recall    F1   AUC
        MLP omnivore recordings     44.26      38.51   92.53 54.39 60.70
Transformer omnivore recordings     42.62      37.14   86.31 51.94 52.11
       LSTM omnivore recordings     45.16      38.43   87.55 53.42 57.48

LSTM ERROR TYPE ANALYSIS
  Technique Error  0  0.00  0.00.1  0.00.2  0.00.3  0.00.4
Preparation Error  0   0.0     0.0     0.0     0.0     0.0
Temperature Error  0   0.0     0.0     0.0     0.0     0.0
Measurement Error  0   0.0     0.0     0.0     0.0     0.0
     Timing Error  0   0.0     0.0     0.0     0.0     0.0
